In [9]:
from utils import *

In [ ]:
#---- download all agencias files from https://www.bcb.gov.br/fis/info/agencias.asp?frame=1
url = 'https://www.bcb.gov.br/fis/info/agencias.asp?frame=1' 
xpath = '/html/body/div/table/tbody/tr[2]/td[2]/form/select'
path = r'C:\Users\gabri\OneDrive\vida_profissional\Projetos\base_dos_dados\br_bcb_agencia\data'

for link in extract_download_links(url = url, xpath=xpath):
    complete_url = 'https://www.bcb.gov.br' + link
    print(complete_url)
    download_and_unzip(url = complete_url,extract_to=path)
    #? if it fails, toggle list reference
    print(f'year {complete_url[23:26]} and month {complete_url[27:29]} downloaded')

In [ ]:
#---- function to check col names for each file

# !to give a bit of context: 
#the files col names have minor sintax changes across the years and months. 
#Also, the col names are not always in the same row.
#The function find_cnpj_row_number is supposed to find the row number of the first occurrence of the CNPJ value (which is always in the first column)
#and then skip all rows before that row.
#the function get_column_names is supposed to read all files in a folder, 
#extract the col names and return a dictionary with the file names as keys and the col names as values.
#It allows to see the pattern of col names across the multiple files

colnames_dict = get_column_names(path)
colnames_dict_count = count_list_values(colnames_dict)

In [ ]:
#read each file and clean it 
dir_path = r'C:\Users\gabri\OneDrive\vida_profissional\Projetos\base_dos_dados\br_bcb_agencia\output'


for file in os.listdir(path):
     # the files format change across the year
    if file.endswith('.xls') or file.endswith('.xlsx'):
        try:
            file_path = os.path.join(path, file)
            df = read_file(file_path=file_path, file_name=file)
            
            #create year and month objects to use in partioning
            ano = df['ano'][1] 
            mes = df['mes'][1]
        
            #general columns stardantization 
            df = clean_column_names(df)
            
            #rename columns
            df.rename(columns= rename_cols(), inplace= True)
            
            # fill left zeros with field range
            df['id_compe_bcb_agencia'] = df['id_compe_bcb_agencia'].astype(str).str.zfill(4)
            df['dv_do_cnpj'] = df['dv_do_cnpj'].astype(str).str.zfill(2)
            df['sequencial_cnpj'] = df['sequencial_cnpj'].astype(str).str.zfill(4)
            df['cnpj'] = df['cnpj'].astype(str).str.zfill(8)
            
            #check existence and create columns
            df = check_and_create_column(df, col_name='data_inicio')
            df = check_and_create_column(df, col_name='instituicao')
            df = check_and_create_column(df, col_name='id_instalacao')
            df = check_and_create_column(df, col_name='id_compe_bcb_instituicao')
            df = check_and_create_column(df, col_name='id_compe_bcb_agencia')


            #some files doesnt have 'id_municipio', just 'nome'.
            #to add new ids is necessary to join by name
            #clean nome municipio
            df = clean_nome_municipio(df)
            #read municipio from bd datalake
            #check if id_municipio already exists
            if 'id_municipio' not in df.columns:
                
                municipio = read_brazilian_municipallity_ids_from_base_dos_dados('pisagab-staging')
                municipio = clean_nome_municipio(municipio)

                df = pd.merge(df,
                municipio[['nome', 'sigla_uf', 'id_municipio']],
                left_on= ['nome','sigla_uf'],
                right_on = ['nome','sigla_uf'],
                how = 'left')

           
            #clean cep column

            # apply the function to the 'cep' column
            df['cep'] = df['cep'].astype(str)
            df['cep'] = df['cep'].apply(remove_non_numeric_chars)
            print('cep ok')

            # cnpj cleaning working
            df = create_cnpj_col(df)
            print(df.columns)
            df['cnpj'] = df['cnpj'].apply(remove_non_numeric_chars)
            df['cnpj'] = df['cnpj'].apply(remove_empty_spaces)
            
            
            # select cols to title
            col_list_to_title = ['endereco', 'complemento' , 'bairro', 'nome_agencia']
            
            for col in col_list_to_title:
                str_to_title(df, column_name = col)
                print(f'column - {col} converted to title')


            #format data_inicio
            df['data_inicio'] =  df['data_inicio'].apply(format_date) 
            
            #strip all df columns
            df = strip_dataframe_columns(df)
            
            #make sure month (mes) is an int to use as partition
            df['mes'] = df['mes'].astype(int)
    
            df = df[order_cols()]

            #create partition
            partition = dir_path + f'/ano={ano}/mes={mes}/'
            
            #create dir if not exists
            if not os.path.exists(partition):
                os.makedirs(partition)
            print(f'dir ano_{ano}-mes_{mes} built')
        
            #save file
            partition = partition + 'agencia.csv'
            df.to_csv(partition, index=False, encoding='utf-8', na_rep='')
            print(f'df ano_{ano}-mes_{mes} saved')
        
            #keep track of possible erros that may occur
        except Exception as e:
            
            error = []
            error.append(f"Error ano{ano}-mes{mes}")
            print(f"Error ano{ano}-mes{mes}")
            print('error: ', e) 
    
#todo: some files have probls in columns 2021 3, 2020-06-01, 2019-12 e 4-1
